# Setting Up Your Jupyter Notebook for Visualization Preparation

## 1. Introduction

**Purpose:** This notebook will prepare and export data sets optimized for creating visualizations in Tableau. The focus is on the significant difference in Paxlovid availability between urban and rural providers.

**Summary of Findings:** Previous analyses revealed a significant disparity in the availability of Paxlovid between urban and rural areas, necessitating targeted visualizations to better understand geographical distributions and potential biases in treatment accessibility.

## 2. Setup and Configuration

### Import Libraries

In [ ]:
import pandas as pd
import sqlalchemy as sa
import matplotlib.pyplot as plt

# Database Connection

In [ ]:
DATABASE_URL = "postgresql://abelshakespeare:@localhost/postgres"
engine = sa.create_engine(DATABASE_URL)
connection = engine.connect()

# 3. Data Retrieval

In [ ]:
query = """
SELECT 
    pi.*,
    pl.*,
    ps.*,
    pa.has_paxlovid,
    pa.has_lagevrio,
    pa.has_veklury,
    zu.is_urban,  -- This is critical
    CASE
        WHEN pa.has_paxlovid THEN 'Paxlovid'
        WHEN pa.has_lagevrio THEN 'Lagevrio'
        WHEN pa.has_veklury THEN 'Veklury'
        ELSE 'No Treatment'
    END as treatment_type
FROM 
    provider_info pi
JOIN 
    provider_location pl ON pi.provider_id = pl.provider_id
JOIN 
    provider_services ps ON pi.provider_id = ps.provider_id
JOIN 
    product_availability pa ON pi.provider_id = pa.provider_id
LEFT JOIN 
    zipcode_urbanity zu ON pl.zip = zu.zip;

"""
df = pd.read_sql(query, connection)

## 4. Enhancing Data for Visualization

### Categorize Urban vs. Rural

In [ ]:
df['location_type'] = df['is_urban'].apply(lambda x: 'Urban' if x else 'Rural')


# Calculate availability rates for Paxlovid, Lagevrio, and Veklury


In [ ]:
df['paxlovid_available'] = df['has_paxlovid'].apply(lambda x: 'Available' if x else 'Not Available')
df['lagevrio_available'] = df['has_lagevrio'].apply(lambda x: 'Available' if x else 'Not Available')
df['veklury_available'] = df['has_veklury'].apply(lambda x: 'Available' if x else 'Not Available')

# Aggregate data to show availability rates of each treatment by state and location type

In [ ]:
agg_df = df.groupby(['state', 'location_type']).agg(
    paxlovid_rate=pd.NamedAgg(column='has_paxlovid', aggfunc='mean'),
    lagevrio_rate=pd.NamedAgg(column='has_lagevrio', aggfunc='mean'),
    veklury_rate=pd.NamedAgg(column='has_veklury', aggfunc='mean')
).reset_index()
agg_df.rename(columns={'paxlovid_rate': 'Paxlovid Availability Rate', 
                       'lagevrio_rate': 'Lagevrio Availability Rate', 
                       'veklury_rate': 'Veklury Availability Rate'}, inplace=True)

# Ensure latitude and longitude are formatted correctly for mapping in Tableau


In [ ]:
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)

# 5. Exporting Data for Tableau

## Export to CSV

In [ ]:
df.to_csv('full_data_for_tableau.csv', index=False)
agg_df.to_csv('aggregated_data_for_tableau.csv', index=False)
